In [1]:
import pandas as pd
import os
import warnings
import settings

warnings.filterwarnings("ignore")

supplier_df = pd.read_pickle(os.path.join(settings.ROOT, "data", "supplier_df.pkl"))
category_df = pd.read_pickle(os.path.join(settings.ROOT, "data", "category_df.pkl"))
mapping_df = pd.read_pickle(os.path.join(settings.ROOT, "data", "mapping_df.pkl"))

mapping_df = mapping_df[['Артикул', 'type_id', 'description_category_id']]
mapping_df.drop_duplicates(inplace=True)

vcdf = mapping_df['Артикул'].value_counts()
vcdf = vcdf[vcdf==1]
articuls = vcdf.index.tolist()

clean_mapping_df = mapping_df[mapping_df['Артикул'].isin(articuls)]

clean_supplier_df = supplier_df[supplier_df['Код артикула'].isin(clean_mapping_df['Артикул'])]
clean_supplier_df.loc[:, 'text'] = clean_supplier_df.apply(lambda row: '/'.join(row[['Название', 'Группа товаров', 'Раздел']].tolist()), axis=1)
clean_supplier_df.rename(columns={'Код артикула':'articul'}, inplace=True)

prepare_supplier_df = clean_supplier_df[['articul', 'text']]
if prepare_supplier_df.shape[0] != clean_supplier_df.shape[0]:
  print('ERROR: не все товары были найдены у поставщика')

def get_category_text(v) -> str | None:
    type_id = v['type_id']
    description_category_id = v['description_category_id']
    filtered_category_df = category_df[((category_df['2_type_id'] == type_id) & (category_df['1_description_category_id'] == description_category_id))]
    if filtered_category_df.empty:
      print(f'NOT FOUND ERROR: {type_id}')
      return None
    elif filtered_category_df.shape[0] > 1:
      print(f'NOT UNIQUE ERROR: {type_id}')
    else:
      return '/'.join(filtered_category_df.iloc[0][['0_category_name', '1_category_name', '2_type_name']].tolist())

data = pd.DataFrame({
    'text': clean_mapping_df.merge(right=prepare_supplier_df, left_on='Артикул', right_on='articul', how='left')['text'].tolist(),
    'label': clean_mapping_df[['description_category_id', 'type_id']].apply(get_category_text, axis=1).tolist()
})

def func(t):
    t = t.split('/')
    return {'text': t[0],
            'input_label': '/'.join(t[1:])}
d = pd.DataFrame(data['text'].apply(func).tolist())
d['output_label'] = data['label']
data = d

In [15]:
import pandas as pd
import os
import warnings
import settings
from scripts import my_logging

warnings.filterwarnings("ignore")
log = my_logging.LOGGER(level='debug').get_logger()

# Загружаем данные
log.debug("START: загрузка файлов")
supplier_df = pd.read_pickle(os.path.join(settings.ROOT, "data", "supplier_df.pkl"))
category_df = pd.read_pickle(os.path.join(settings.ROOT, "data", "category_df.pkl"))
mapping_df = pd.read_pickle(os.path.join(settings.ROOT, "data", "mapping_df.pkl"))
log.info("DONE: загрузка файлов")

# Убираем дубликаты и фильтруем строки с уникальными артикулами
log.debug("START: предварительная обратка")
mapping_df = mapping_df[['Артикул', 'type_id', 'description_category_id']].drop_duplicates()
articuls = mapping_df['Артикул'].unique()
log.debug("DONE: Получение уникальных артикулов")

# Фильтруем supplier_df на основе уникальных артикулов
clean_supplier_df = supplier_df[supplier_df['Код артикула'].isin(articuls)].copy()
clean_supplier_df['text'] = clean_supplier_df['Название']
clean_supplier_df['label'] = clean_supplier_df[['Группа товаров', 'Раздел']].agg('/'.join, axis=1)
prepare_supplier_df = clean_supplier_df[['Код артикула', 'text', 'label']]
log.debug("DONE: Получение среза данных поставщика по артикулам с известными категориями")

# Проверяем корректность
if len(articuls) != len(prepare_supplier_df):
    log.warning("Не все товары были найдены у поставщика")

# Функция для получения категорий
def get_category_text(row):
    filtered = category_df[(category_df['2_type_id'] == row['type_id']) & 
                           (category_df['1_description_category_id'] == row['description_category_id'])]
    if filtered.shape[0] == 1:
        return '/'.join(filtered.iloc[0][['0_category_name', '1_category_name', '2_type_name']].tolist())
    elif filtered.empty:
        log.warning(f'NOT FOUND: {row["Артикул"]} --- {row["type_id"]}')
        return None
    else:
        log.warning(f'NOT UNIQUE: {row["Артикул"]} --- {row["type_id"]}')
        return None

# Собираем данные
merged_df = mapping_df.merge(prepare_supplier_df, left_on='Артикул', right_on='Код артикула', how='left')
log.debug("DONE: Объединение с известными метками категорий")
data = pd.DataFrame({
    'text': merged_df['text'],
    'input_label': merged_df['label'],
    'output_label': clean_mapping_df.apply(get_category_text, axis=1)
})
log.info("DONE: Формирование тренировочного датасета")

DEBUG    | 02-09-2024 18:30:52 | 11 - C:\Users\Пользователь\AppData\Local\Temp\ipykernel_12952\2616341423.py | START: загрузка файлов
DEBUG    | 02-09-2024 18:30:52 | 11 - C:\Users\Пользователь\AppData\Local\Temp\ipykernel_12952\2616341423.py | START: загрузка файлов
INFO     | 02-09-2024 18:30:53 | 15 - C:\Users\Пользователь\AppData\Local\Temp\ipykernel_12952\2616341423.py | DONE: загрузка файлов
INFO     | 02-09-2024 18:30:53 | 15 - C:\Users\Пользователь\AppData\Local\Temp\ipykernel_12952\2616341423.py | DONE: загрузка файлов
DEBUG    | 02-09-2024 18:30:53 | 18 - C:\Users\Пользователь\AppData\Local\Temp\ipykernel_12952\2616341423.py | START: предварительная обратка
DEBUG    | 02-09-2024 18:30:53 | 18 - C:\Users\Пользователь\AppData\Local\Temp\ipykernel_12952\2616341423.py | START: предварительная обратка
DEBUG    | 02-09-2024 18:30:53 | 21 - C:\Users\Пользователь\AppData\Local\Temp\ipykernel_12952\2616341423.py | DONE: Получение уникальных артикулов
DEBUG    | 02-09-2024 18:30:53 | 2

In [16]:
clean_mapping_df

,Артикул,type_id,description_category_id
0,89160,93864,72163739
1,89161,93864,72163739
2,89162,93864,72163739
3,89168,93864,72163739
4,89164,93872,72163739
...,...,...,...
3357,13396,92101,17028653
3358,15766,92082,17028653
3359,85831,92156,17028653
3360,85832,92156,17028653


In [14]:
articuls

Index([85577, 85583, 85574, 85584, 85573, 80182, 86018, 85586, 86011, 84615,
       ...
       62283, 62284, 67594, 62297, 62298, 62300, 64390, 64392, 64393, 85833],
      dtype='int64', name='Артикул', length=3010)